In [90]:
import os
import pandas as pd
import seaborn as sns
from pathlib import Path
import numpy as np

def set_correct_working_dir(working_dir: str):
    current_working_dir = Path(os.getcwd())
    if working_dir == current_working_dir.name:
        print('WD correct')
        return
    for directory in current_working_dir.parents:
        if working_dir == directory.name:
            os.chdir(str(directory.absolute()))
            print('New WD:', str(directory.absolute()))
            return
    raise FileNotFoundError

set_correct_working_dir('geo-locations')

WD correct


In [91]:
country_path = Path(os.getcwd()) / 'data' / 'che' 
source = country_path / 'source' / 'PLZO_CSV_WGS84' / 'PLZO_CSV_WGS84.csv'
# column description to be found at: https://download.geonames.org/export/zip/
ch_official_data = pd.read_csv(source, sep=';', encoding='latin_1')
display(ch_official_data.head())

,Ortschaftsname,PLZ,Zusatzziffer,Gemeindename,BFS-Nr,Kantonskürzel,E,N,Sprache
0,Aeugst am Albis,8914,0,Aeugst am Albis,1,ZH,8.488313,47.267004,de
1,Aeugstertal,8914,2,Aeugst am Albis,1,ZH,8.493642,47.282761,de
2,Zwillikon,8909,0,Affoltern am Albis,2,ZH,8.431459,47.287633,de
3,Affoltern am Albis,8910,0,Affoltern am Albis,2,ZH,8.448945,47.279169,de
4,Bonstetten,8906,0,Bonstetten,3,ZH,8.467611,47.315510,de


In [92]:
ch_official_data_reduced = ch_official_data[['PLZ', 'Gemeindename', 'Kantonskürzel', 'E', 'N']].copy()
ch_official_data_reduced.rename(columns={'PLZ': 'postal_code', 'Gemeindename': 'town', 'Kantonskürzel': 'region', 'E': 'latitude', 'N': 'longitude'}, inplace=True)
display(ch_official_data_reduced.head())
ch_official_data_reduced.drop_duplicates(inplace=True)
ch_official_data_reduced = ch_official_data_reduced.groupby('postal_code').agg({'town': lambda col: ','.join(col),
                                                                                'region': lambda col2: ','.join(col2), 
                                                                                'latitude': 'mean',
                                                                                'longitude': 'mean'}).reset_index()
ch_official_data_reduced['town'] = ch_official_data_reduced['town'].apply(lambda x: ','.join(list(set(x.split(',')))) if ',' in x else x)
ch_official_data_reduced['region'] = ch_official_data_reduced['region'].apply(lambda x: ','.join(list(set(x.split(',')))) if ',' in x else x)
ch_official_data_reduced.reset_index(inplace=True, drop=True)
ch_official_data_reduced.sort_values('postal_code', inplace=True)
ch_official_data_reduced.insert(loc=0, column='country_code', value='che')
display(ch_official_data_reduced.head())

,postal_code,town,region,latitude,longitude
0,8914,Aeugst am Albis,ZH,8.488313,47.267004
1,8914,Aeugst am Albis,ZH,8.493642,47.282761
2,8909,Affoltern am Albis,ZH,8.431459,47.287633
3,8910,Affoltern am Albis,ZH,8.448945,47.279169
4,8906,Bonstetten,ZH,8.467611,47.315510


,country_code,postal_code,town,region,latitude,longitude
0,che,1000,Lausanne,VD,6.687041,46.553488
1,che,1003,Lausanne,VD,6.630034,46.520004
2,che,1004,Lausanne,VD,6.618678,46.528480
3,che,1005,Lausanne,VD,6.642500,46.519859
4,che,1006,Lausanne,VD,6.637110,46.510849


In [93]:
file_path = country_path / 'che_geocoding.csv'
ch_official_data_reduced.to_csv(file_path, index=False)